In [5]:
import pandas as pd

In [75]:
full_channel_df = pd.read_json('workspace/BTC_FullChannel_GDAX_20221128_16hr.json', lines=True)

In [76]:
ticker_df = pd.read_json('workspace/BTC_Ticker_GDAX_20221128_16hr.json', lines=True)

In [77]:
full_channel_df.columns

Index(['order_id', 'reason', 'price', 'remaining_size', 'type', 'side',
       'product_id', 'time', 'sequence', 'order_type', 'size', 'client_oid',
       'trade_id', 'maker_order_id', 'taker_order_id', 'funds', 'old_size',
       'new_size'],
      dtype='object')

In [94]:
full_channel_df.time = pd.to_datetime(full_channel_df.time)
full_channel_df['timestamp'] = full_channel_df['time'].astype('int64')

ticker_df.time = pd.to_datetime(ticker_df.time)
ticker_df['timestamp'] = ticker_df['time'].astype('int64')

In [95]:
full_channel_df = full_channel_df.sort_values(by='timestamp')
ticker_df = ticker_df.sort_values(by='timestamp')

In [80]:
full_channel_df.time.describe()

count                                2742118
mean     2022-11-28 16:29:28.831980032+00:00
min         2022-11-28 15:59:44.575431+00:00
25%      2022-11-28 16:16:00.510982912+00:00
50%      2022-11-28 16:27:55.734088448+00:00
75%      2022-11-28 16:44:48.903901696+00:00
max         2022-11-28 16:57:32.297459+00:00
Name: time, dtype: object

In [81]:
full_channel_df.count()

order_id          2696392
reason             912060
price             2741488
remaining_size    1785146
type              2742118
side              2742118
product_id        2742118
time              2742118
sequence          2742118
order_type         911241
size               956746
client_oid         911241
trade_id            45726
maker_order_id      45726
taker_order_id      45726
funds                 238
old_size                5
new_size                5
timestamp         2742118
dtype: int64

In [82]:
full_channel_df.head()

,order_id,reason,price,remaining_size,type,side,product_id,time,sequence,order_type,size,client_oid,trade_id,maker_order_id,taker_order_id,funds,old_size,new_size,timestamp
0,f8bd540c-dce0-4d73-9d83-bc3fa8e16d2d,canceled,16124.00,0.049615,done,buy,BTC-USD,2022-11-28 15:59:44.575431+00:00,50873884377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1669651184575431000
1,ea74cf26-60ef-4169-b033-0c9b20aade90,NaN,16135.12,NaN,received,buy,BTC-USD,2022-11-28 15:59:44.581420+00:00,50873884378,limit,0.04307,20221128-0000-0000-0002-000000045869,NaN,NaN,NaN,NaN,NaN,NaN,1669651184581420000
2,NaN,NaN,16135.12,NaN,match,sell,BTC-USD,2022-11-28 15:59:44.581420+00:00,50873884379,NaN,0.04307,NaN,462208805.0,d5648948-e020-42ef-98d2-02ceeb505d0c,ea74cf26-60ef-4169-b033-0c9b20aade90,NaN,NaN,NaN,1669651184581420000
3,ea74cf26-60ef-4169-b033-0c9b20aade90,filled,16135.12,0.000000,done,buy,BTC-USD,2022-11-28 15:59:44.581420+00:00,50873884380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1669651184581420000
4,3c3295e1-5201-45d1-969d-f252396d81d2,NaN,16133.41,NaN,received,buy,BTC-USD,2022-11-28 15:59:44.585097+00:00,50873884381,limit,0.25000,53685441-1c9c-4def-9877-90beae502b3f,NaN,NaN,NaN,NaN,NaN,NaN,1669651184585097000


In [83]:
full_channel_df.timestamp.head()

0    1669651184575431000
1    1669651184581420000
2    1669651184581420000
3    1669651184581420000
4    1669651184585097000
Name: timestamp, dtype: int64

In [93]:
ticker_df.columns

Index(['type', 'sequence', 'product_id', 'price', 'open_24h', 'volume_24h',
       'low_24h', 'high_24h', 'volume_30d', 'best_bid', 'best_bid_size',
       'best_ask', 'best_ask_size', 'side', 'time', 'trade_id', 'last_size'],
      dtype='object')

In [96]:
ticker_df.head()

,type,sequence,product_id,price,open_24h,volume_24h,low_24h,high_24h,volume_30d,best_bid,best_bid_size,best_ask,best_ask_size,side,time,trade_id,last_size,timestamp
0,ticker,50873887939,BTC-USD,16135.80,16564.49,23874.549553,16044.99,16588.46,1.053822e+06,16134.97,0.029699,16135.80,0.025891,buy,2022-11-28 15:59:54.510812+00:00,462208903,0.000056,1669651194510812000
1,ticker,50873887955,BTC-USD,16135.79,16564.49,23874.551463,16044.99,16588.46,1.053822e+06,16134.97,0.029699,16135.79,0.000001,buy,2022-11-28 15:59:54.553491+00:00,462208904,0.001910,1669651194553491000
2,ticker,50873887963,BTC-USD,16135.79,16564.49,23874.551464,16044.99,16588.46,1.053822e+06,16134.97,0.029699,16135.80,0.025891,buy,2022-11-28 15:59:54.566656+00:00,462208905,0.000001,1669651194566656000
3,ticker,50873887965,BTC-USD,16135.80,16564.49,23874.577353,16044.99,16588.46,1.053822e+06,16134.97,0.029699,16135.80,0.000002,buy,2022-11-28 15:59:54.566656+00:00,462208906,0.025889,1669651194566656000
4,ticker,50873888219,BTC-USD,16136.67,16564.49,23874.579264,16044.99,16588.46,1.053822e+06,16135.46,0.003563,16136.67,0.015000,buy,2022-11-28 15:59:55.037482+00:00,462208907,0.001911,1669651195037482000


In [118]:
price_df = ticker_df[['timestamp', 'price', 'best_bid', 'best_ask', 'time']].copy()
price_df['mid_price'] = (price_df['best_bid'] + price_df['best_ask']) / 2

In [114]:
delta_t_ns = 1_000_000_000
start_t_ns = price_df.timestamp.min()
end_t_ns = price_df.timestamp.max()

In [120]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 524.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 2.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 37.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.4 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 32.5 MB/s eta 0:00:0000:01:00:01


In [ ]:
from matplotlib import pyplot as plt

def 

In [119]:
for t in range(start_t_ns, end_t_ns, delta_t_ns):
    mask = price_df.timestamp.between(t, t + delta_t_ns, inclusive='left')
    tmp_df = price_df[mask]
    print(tmp_df.head())
    break


             timestamp     price  best_bid  best_ask  \
0  1669651194510812000  16135.80  16134.97  16135.80   
1  1669651194553491000  16135.79  16134.97  16135.79   
2  1669651194566656000  16135.79  16134.97  16135.80   
3  1669651194566656000  16135.80  16134.97  16135.80   
4  1669651195037482000  16136.67  16135.46  16136.67   

                              time  mid_price  
0 2022-11-28 15:59:54.510812+00:00  16135.385  
1 2022-11-28 15:59:54.553491+00:00  16135.380  
2 2022-11-28 15:59:54.566656+00:00  16135.385  
3 2022-11-28 15:59:54.566656+00:00  16135.385  
4 2022-11-28 15:59:55.037482+00:00  16136.065  
